In [ ]:
import time
import torch
import numpy as np
from transformers import ViTModel, ViTConfig

from vit.vit import VIT
from vit.utils import transfer_pretrained_weights

In [ ]:
torch.manual_seed(0)
# torch.use_deterministic_algorithms(True)
torch.backends.cudnn.benchmark = False

def diff(a, b):
    return torch.abs(a - b).mean()

device = 'cuda:0'
dtype = torch.float32
model_id = 'google/vit-base-patch16-224'
vit_config = ViTConfig(model_id)

**Loading weights**

In [ ]:
height, width, channels = vit_config.image_size, vit_config.image_size, vit_config.num_channels
patch_size = vit_config.patch_size
hidden_dim = 768
num_heads=vit_config.num_attention_heads
num_layers=vit_config.num_hidden_layers

model = VIT(
    height=height,
    width=width,
    channels=channels,
    patch_size=patch_size,
    hidden_dim=hidden_dim,
    num_heads=num_heads,
    num_layers=num_layers
)
model.to(device, dtype)

pretrained_model = ViTModel.from_pretrained(model_id, add_pooling_layer=False)
pretrained_model.to(device, dtype)
pretrained_model.eval()

model = transfer_pretrained_weights(
    pretrained_model=pretrained_model,
    custom_model=model
)

sum(p.numel() for p in pretrained_model.parameters()), sum(p.numel() for p in model.parameters())

In [ ]:
pretrained_model

In [ ]:
model

**Verifying layer by layer outputs**

In [ ]:
store_pretrained = {}
store_custom = {}

def hook(module, input, output, name, store):
    store[name] = output

for name, layer in pretrained_model.named_modules():
    layer.register_forward_hook(lambda layer, input, output, name=name: hook(layer, input, output, name, store_pretrained))

for name, layer in model.named_modules():
    layer.register_forward_hook(lambda layer, input, output, name=name: hook(layer, input, output, name, store_custom))

In [ ]:
inputs = torch.randn((1, 3, 224, 224)).to(device, dtype)

with torch.no_grad():
    pretrained_out = pretrained_model(inputs)[0]
    custom_out = model(inputs)

In [ ]:
weight_mapping = {
    'embeddings.patch_embeddings.projection': 'embeddings.projection',
    'embeddings.patch_embeddings': None,
    'embeddings.dropout': None,
    'embeddings': 'embeddings',
    'layernorm': 'layernorm',
}

for i in range(0, num_heads):
    weight_mapping.update({
        f'encoder.layer.{i}.layernorm_before': f'encoder.layer.{i}.layernorm_before',
        f'encoder.layer.{i}.attention.attention.query': [f'encoder.layer.{i}.attention.attention.0.query', f'encoder.layer.{i}.attention.attention.1.query', f'encoder.layer.{i}.attention.attention.2.query', f'encoder.layer.{i}.attention.attention.3.query', f'encoder.layer.{i}.attention.attention.4.query', f'encoder.layer.{i}.attention.attention.5.query', f'encoder.layer.{i}.attention.attention.6.query', f'encoder.layer.{i}.attention.attention.7.query', f'encoder.layer.{i}.attention.attention.8.query', f'encoder.layer.{i}.attention.attention.9.query', f'encoder.layer.{i}.attention.attention.10.query', f'encoder.layer.{i}.attention.attention.11.query'],
        f'encoder.layer.{i}.attention.attention.key': [f'encoder.layer.{i}.attention.attention.0.key', f'encoder.layer.{i}.attention.attention.1.key', f'encoder.layer.{i}.attention.attention.2.key', f'encoder.layer.{i}.attention.attention.3.key', f'encoder.layer.{i}.attention.attention.4.key', f'encoder.layer.{i}.attention.attention.5.key', f'encoder.layer.{i}.attention.attention.6.key', f'encoder.layer.{i}.attention.attention.7.key', f'encoder.layer.{i}.attention.attention.8.key', f'encoder.layer.{i}.attention.attention.9.key', f'encoder.layer.{i}.attention.attention.10.key', f'encoder.layer.{i}.attention.attention.11.key'],
        f'encoder.layer.{i}.attention.attention.value': [f'encoder.layer.{i}.attention.attention.0.value', f'encoder.layer.{i}.attention.attention.1.value', f'encoder.layer.{i}.attention.attention.2.value', f'encoder.layer.{i}.attention.attention.3.value', f'encoder.layer.{i}.attention.attention.4.value', f'encoder.layer.{i}.attention.attention.5.value', f'encoder.layer.{i}.attention.attention.6.value', f'encoder.layer.{i}.attention.attention.7.value', f'encoder.layer.{i}.attention.attention.8.value', f'encoder.layer.{i}.attention.attention.9.value', f'encoder.layer.{i}.attention.attention.10.value', f'encoder.layer.{i}.attention.attention.11.value'],
        f'encoder.layer.{i}.attention.attention.dropout': None,
        f'encoder.layer.{i}.attention.attention': [f'encoder.layer.{i}.attention.attention.0', f'encoder.layer.{i}.attention.attention.1', f'encoder.layer.{i}.attention.attention.2', f'encoder.layer.{i}.attention.attention.3', f'encoder.layer.{i}.attention.attention.4', f'encoder.layer.{i}.attention.attention.5', f'encoder.layer.{i}.attention.attention.6', f'encoder.layer.{i}.attention.attention.7', f'encoder.layer.{i}.attention.attention.8', f'encoder.layer.{i}.attention.attention.9', f'encoder.layer.{i}.attention.attention.10', f'encoder.layer.{i}.attention.attention.11'],
        f'encoder.layer.{i}.attention.output.dense': None,
        f'encoder.layer.{i}.attention.output.dropout': None,
        f'encoder.layer.{i}.attention.output': f'encoder.layer.{i}.attention.output',
        f'encoder.layer.{i}.attention': f'encoder.layer.{i}.attention',
        f'encoder.layer.{i}.layernorm_after': f'encoder.layer.{i}.layernorm_after',
        f'encoder.layer.{i}.intermediate.dense': f'encoder.layer.{i}.intermediate',
        f'encoder.layer.{i}.intermediate.intermediate_act_fn': None,
        f'encoder.layer.{i}.intermediate': None,
        f'encoder.layer.{i}.output.dense': f'encoder.layer.{i}.output',
        f'encoder.layer.{i}.output.dropout': None,
        f'encoder.layer.{i}.output': None,
        f'encoder.layer.{i}': f'encoder.layer.{i}',
    })

In [ ]:
for k, v in weight_mapping.items():

    if k and v:
        if type(v) == list:
            val2 = torch.cat([store_custom[e] for e in v], dim=-1)
        else:
            val2 = store_custom[v]

        val1 = store_pretrained[k]

        if type(val1) == tuple:
            val1 = val1[0]

        print(f'{k}\t{val1.shape}, {v}\t{val2.shape}\t{torch.abs(val1-val2).max()}')

        match = torch.allclose(val1, val2, rtol=0, atol=1)

        print('\n')

Rough

In [ ]:
i1 = store_pretrained['encoder.layer.5.intermediate.dense']
i2 = store_custom['encoder.layer.5.intermediate']

w1 = pretrained_model.encoder.layer[5].output.dense.weight
b1 = pretrained_model.encoder.layer[5].output.dense.bias

w2 = model.encoder.layer[5].output.weight
b2 = model.encoder.layer[5].output.bias

z1 = torch.nn.functional.gelu(i1)

# For i1, apply layernorm and then mult by 

v1 = store_pretrained['encoder.layer.5.output.dense']
v2 = store_custom['encoder.layer.5.output']

In [ ]:
diff(z1, i2), diff(w1.T, w2), diff(b1, b2), diff(v1, v2)

In [ ]:
o1 = torch.matmul(z1, w1.T) + b1
diff(v1, o1)

In [ ]:
from vit.kernels.matmul import matmul_triton

In [ ]:
w2 = torch.ones_like(w2)
b2 = torch.ones_like(b2)

o2 = torch.matmul(i2, w2) + b2
o2_hat = matmul_triton(i2, w2, b2)

# diff(v2, o2), diff(v1, o2), diff(v2, o2_hat), diff(o2, o2_hat), diff(v1, o2_hat)

diff(o2, o2_hat)

**Assigning identity weights for debugging**

In [ ]:
pretrained_state_dict = pretrained_model.state_dict()
custom_state_dict = model.state_dict()

In [ ]:
pretrained_state_dict_new = {}
custom_state_dict_new = {}

for k, v in pretrained_state_dict.items():
    pretrained_state_dict_new[k] = torch.ones_like(v).to(device, dtype)

for k, v in custom_state_dict.items():
    custom_state_dict_new[k] = torch.ones_like(v).to(device, dtype)

In [ ]:
model.load_state_dict(custom_state_dict_new)
pretrained_model.load_state_dict(pretrained_state_dict_new)

**Benchmarking**

In [ ]:
from vit.utils import benchmark

batch_sizes = [1, 2, 4]
hf_time, custom_time = benchmark(pretrained_model, model, batch_sizes=batch_sizes)

In [ ]:
inputs = torch.randn(32, 3, 224, 224).to(device, dtype)

In [ ]:
for i in range(50):

    with torch.no_grad():
        pretrained_out = pretrained_model(inputs)[0]
        custom_out = model(inputs)

In [ ]:
for i in range(100):
    m1_times = []
    m2_times = []
    with torch.no_grad():
        start_time = time.time()
        pretrained_out = pretrained_model(inputs)[0]
        m1_time = time.time() - start_time

        start_time = time.time()
        custom_out = model(inputs)
        m2_time = time.time() - start_time

        m1_times.append(m1_time)
        m2_times.append(m2_time)

In [ ]:
np.mean(m1_times), np.mean(m2_times)